# Pull PPC Polygons from TerraMatch API

This notebook sets up the process to pull PPC polygon geometries and metadata from the TerraMatch API.

In [1]:
import yaml
import pandas as pd
from tm_api_utils import pull_tm_api_data, patch_tm_api_data
from tqdm import tqdm
import json
import sys
from datetime import datetime
sys.path.append('../src/')
import api_utils as api
import process_tm_api_results as clean
import geospatial_utils_NEW as geo

## Set file paths

In [2]:
# Naming convention
run_name = 'ppc_batch1'
run_dir = 'ppc_batch1'

# Today's date
today = datetime.today().strftime('%Y-%m-%d') # Check computer date before running

## Input Files
# List of all approved projects on TerraMatch
approved_projects_file = '../projects_all_approved_202502211226.csv'

# Batch 1 Projects
batch_1_file = '/home/darby/github_repos/tf-biophysical-monitoring/data/ppc/ppc_batch1_projects_20250501.csv'

## Output Files
# A JSON file that stores the results of the TM API pull; we'll read it back in to clean the results (outfile, infile)
tm_api_pull_results_file = f'/home/darby/github_repos/tf-biophysical-monitoring/data/ppc/tm_api_response_prod_{run_name}_{today}.json'

# The cleaned polygon features csv
polygon_features_file = f'/home/darby/github_repos/tf-biophysical-monitoring/data/ppc/tm_api_{run_name}_{today}.csv' 

## Read in files

In [3]:
# List of all approved projects on TerraMatch
approved_projects_df = pd.read_csv(approved_projects_file)

# Batch 1 Projects
batch_1_df = pd.read_csv(batch_1_file)

## Set up token and API URL

In [4]:
# Set up token access
auth_path = '../secrets.yaml'
with open(auth_path) as auth_file:
    auth = yaml.safe_load(auth_file)
headers = {
    'Authorization': f"Bearer {auth['access_token']}"
    }

In [5]:
# TerraMatch API URLs
staging_url = "https://api-staging.terramatch.org/research/v3/sitePolygons?" # Use for testing queries
prod_url = "https://api.terramatch.org/research/v3/sitePolygons?" # Use to pull data for analysis

## Create list of projects to pull

In [6]:
# Make a list of the unique project_ids from the Batch 1 projects
batch_1_proj_ids = list(batch_1_df.project_id.unique())

# Filter the list of all approved projects by cohort ('ppc'), project_id (the Batch 1 list), 
ppc = approved_projects_df[approved_projects_df['cohort'] == 'ppc']

# Filter to just the batch 1 projects list
batch1 = ppc[ppc['project_id'].isin(batch_1_proj_ids)]

In [7]:
batch_1_proj_ids

['244eaf7e-e109-47b2-b84e-9ebe24508391',
 '24d8c9a2-b8ef-481c-930b-78c9aeaf239e',
 'f17dd6cf-8187-4edd-895e-07013d4990c9',
 '1115dda6-0165-4099-b52f-0ac53595c3a9',
 '465f543e-d53a-4356-ae8d-9790aa42d30e',
 'ad149677-7ee0-479c-8d23-aa8c3bf58532',
 '1977b649-908c-46c3-836d-f4f6485427c2',
 '6d9089aa-2a6f-4dc0-8064-32c5b67ffed6']

In [8]:
batch_1_df

,project_name,project_year,project_id,country,notes
0,EMA - Maranhao (Amazon),2022,244eaf7e-e109-47b2-b84e-9ebe24508391,BR,NaN
1,Puerto Princesa Forest Restoration,2023,24d8c9a2-b8ef-481c-930b-78c9aeaf239e,PH,NaN
2,Puerto Princesa Forest Restoration,2022,24d8c9a2-b8ef-481c-930b-78c9aeaf239e,PH,NaN
3,Restoring mangrove ecosystems in the UAE,2024,f17dd6cf-8187-4edd-895e-07013d4990c9,AE,NaN
4,Restoring mangrove ecosystems in the UAE,2023,f17dd6cf-8187-4edd-895e-07013d4990c9,AE,NaN
5,Restoring mangrove ecosystems in the UAE,2022,f17dd6cf-8187-4edd-895e-07013d4990c9,AE,NaN
6,GANB - Bahia (Atlantic Forest),2021,1115dda6-0165-4099-b52f-0ac53595c3a9,BR,NaN
7,Ciclos - Flagship,2023,465f543e-d53a-4356-ae8d-9790aa42d30e,BR,NaN
8,Trees for Villagers and Wildlife Jharkhand,2023,ad149677-7ee0-479c-8d23-aa8c3bf58532,IN,NaN
9,CEPAN - Flagship,2024,1977b649-908c-46c3-836d-f4f6485427c2,BR,NaN


In [9]:
batch_1_proj_ids

['244eaf7e-e109-47b2-b84e-9ebe24508391',
 '24d8c9a2-b8ef-481c-930b-78c9aeaf239e',
 'f17dd6cf-8187-4edd-895e-07013d4990c9',
 '1115dda6-0165-4099-b52f-0ac53595c3a9',
 '465f543e-d53a-4356-ae8d-9790aa42d30e',
 'ad149677-7ee0-479c-8d23-aa8c3bf58532',
 '1977b649-908c-46c3-836d-f4f6485427c2',
 '6d9089aa-2a6f-4dc0-8064-32c5b67ffed6']

In [10]:
ids = batch_1_proj_ids
ids = [#'244eaf7e-e109-47b2-b84e-9ebe24508391',
       '24d8c9a2-b8ef-481c-930b-78c9aeaf239e', 
       'f17dd6cf-8187-4edd-895e-07013d4990c9', 
       '1115dda6-0165-4099-b52f-0ac53595c3a9', 
       '465f543e-d53a-4356-ae8d-9790aa42d30e',
       '1977b649-908c-46c3-836d-f4f6485427c2']
ids = ['36504a4e-f7a3-4963-9ff2-9aa9982cf990', '465f543e-d53a-4356-ae8d-9790aa42d30e']

In [11]:
ids

['36504a4e-f7a3-4963-9ff2-9aa9982cf990',
 '465f543e-d53a-4356-ae8d-9790aa42d30e']

## Pull polygons from TM API

In [12]:
results = api.pull_wrapper(prod_url, headers, ids, outfile=tm_api_pull_results_file)

Pulling Projects: 100%|██████████| 2/2 [00:33<00:00, 16.68s/project]


Results saved to /home/darby/github_repos/tf-biophysical-monitoring/data/ppc/tm_api_response_prod_ppc_batch1_2025-05-02.json


In [13]:
df = pd.DataFrame(results)

In [14]:
print(len(df))
#print(f"df has {df.project_id.nunique()} unique projects")
#print(f"df has {df.poly_id.nunique()} unique polygons")
#df['project_id'].value_counts()
df.head()

1528


,status,plantStart,calcArea,plantEnd,practice,targetSys,distr,numTrees,name,siteId,projectId,indicators,siteName,geometry,establishmentTreeSpecies,reportingPeriods,lightResource,poly_id,project_id
0,approved,2023-03-03,0.365172,None,tree-planting,agroforest,None,NaN,KE0123 (new),9d0e5c0a-aba6-435f-b77c-6ca0d8f6b791,36504a4e-f7a3-4963-9ff2-9aa9982cf990,"[{'indicatorSlug': 'restorationByStrategy', 'y...",Bweyeye-Butare,"{'type': 'Polygon', 'coordinates': [[[29.16067...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",False,720bb5c6-bd90-4dd4-a4cf-2de6ebce8888,36504a4e-f7a3-4963-9ff2-9aa9982cf990
1,approved,2023-11-03,0.116778,None,tree-planting,agroforest,None,NaN,KE0127 (new),9d0e5c0a-aba6-435f-b77c-6ca0d8f6b791,36504a4e-f7a3-4963-9ff2-9aa9982cf990,"[{'indicatorSlug': 'restorationByStrategy', 'y...",Bweyeye-Butare,"{'type': 'Polygon', 'coordinates': [[[29.15960...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",False,be5eaf1e-b0d5-403c-9e62-b877dee18ea0,36504a4e-f7a3-4963-9ff2-9aa9982cf990
2,approved,2023-03-03,0.691817,None,tree-planting,agroforest,None,NaN,KA0023 (new),9d0e5c0a-aba6-435f-b77c-6ca0d8f6b791,36504a4e-f7a3-4963-9ff2-9aa9982cf990,"[{'indicatorSlug': 'restorationByStrategy', 'y...",Bweyeye-Butare,"{'type': 'Polygon', 'coordinates': [[[29.17034...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",False,96ae8767-7687-4533-b2f8-31fb571149db,36504a4e-f7a3-4963-9ff2-9aa9982cf990
3,approved,2023-11-03,0.256205,None,tree-planting,agroforest,None,NaN,KE0124 (new),9d0e5c0a-aba6-435f-b77c-6ca0d8f6b791,36504a4e-f7a3-4963-9ff2-9aa9982cf990,"[{'indicatorSlug': 'restorationByStrategy', 'y...",Bweyeye-Butare,"{'type': 'Polygon', 'coordinates': [[[29.16024...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",False,128b9266-cac4-407a-86c2-49b6749fd7b7,36504a4e-f7a3-4963-9ff2-9aa9982cf990
4,approved,2023-02-26,0.236050,None,tree-planting,agroforest,None,NaN,KE0138 (new),9d0e5c0a-aba6-435f-b77c-6ca0d8f6b791,36504a4e-f7a3-4963-9ff2-9aa9982cf990,"[{'indicatorSlug': 'restorationByStrategy', 'y...",Bweyeye-Butare,"{'type': 'Polygon', 'coordinates': [[[29.16007...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",False,0415a328-8634-4007-9159-9ce9d7c331a4,36504a4e-f7a3-4963-9ff2-9aa9982cf990
